In [1]:
import pyarrow
import pandas as pd
import numpy as np
import os

# use paraquet to load the data as-is
# Define the target directory path
parquet_dir = "../data/processed/"

# Create the directory if it doesn't exist
os.makedirs(parquet_dir, exist_ok=True)  # `exist_ok=True` prevents errors if dir already exists
df = pd.read_parquet(os.path.join(parquet_dir, "metadata.parquet"))


In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1866 entries, 0 to 1865
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   message_id        1866 non-null   int64              
 1   channel_name      1866 non-null   object             
 2   channel_username  1866 non-null   object             
 3   timestamp         1866 non-null   datetime64[ns, UTC]
 4   has_media         1866 non-null   bool               
 5   media_type        1866 non-null   object             
 6   message_length    1866 non-null   int64              
 7   word_count        1866 non-null   int64              
 8   amharic_ratio     1866 non-null   float64            
 9   hour_of_day       1866 non-null   int32              
 10  day_of_week       1866 non-null   object             
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), int32(1), int64(3), object(4)
memory usage: 140.4+ KB


In [ ]:
# Load your actual metadata dataset
def load_vendor_data(metadata_df, vendor_username):
    """
    Load data for a specific vendor from your metadata DataFrame
    """
    vendor_df = metadata_df[metadata_df['channel_username'] == vendor_username].copy()
    return vendor_df